In [266]:
import simpy
import random
import math
import numpy

In [267]:
RANDOM_SEED = 9780

INTERARRIVAL_MEAN = 6 
INTERARRIVAL_RATE = 1.0 / INTERARRIVAL_MEAN

CALL_MEAN = 5 
CALL_RATE = 1.0 / CALL_MEAN


SERVICE_TIME_OPERATOR1_MEAN = 12
SERVICE_TIME_OPERATOR_VARIENCE = 36
mu= math.log(SERVICE_TIME_OPERATOR1_MEAN**2/math.sqrt(SERVICE_TIME_OPERATOR_VARIENCE+SERVICE_TIME_OPERATOR1_MEAN**2))
sigma=math.sqrt(math.log(1+SERVICE_TIME_OPERATOR_VARIENCE /SERVICE_TIME_OPERATOR1_MEAN))


SERVICE_TIME_OPERATOR2_MEAN = 4
SERVICE_TIME_OPERATOR2_DEVIATION = math.sqrt(3)

BREAK_TIME = 3
BREAK_TIME_RATE = 1/60


CUSTOMER_COUNT = 1000
random.seed(RANDOM_SEED)

In [268]:
# Duration of the conversation between the customer and the operator (Operator1 service time)
service_times_operator1 = []

# Duration of the conversation between the customer and the operator2 (operator2 service time)
service_times_operator2 = []

# Time spent by a customer while it waits for the operator1 (Operator1 Queue waiting time)
queue_w_times_operator1 = []

# Time spent by a customer while it waits for the operator2 (operator2 Queue waiting time)
queue_w_times_operator2 = []

# The number of customer in the answering system
queue_w_customer = []

# The number of customer in the answering system
queue_w_operator1 = []

# The number of customer in the answering system
queue_w_operator2 = []



# Simulation end time
end_time = 0;



In [269]:
class Customer(object):
    def __init__(self, name, env, operator1, operator2):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
    
    def call(self):
        global end_time 
        
        print('%s initiated a call at %g' % (self.name, self.env.now))
        queue_w_customer.append(self.env.now - self.arrival_t)
        
        prob=random.uniform(0,10)
        print('prob',prob)
        error=random.uniform(0,10)
        print('error',error)
        print('inital len',len(queue_w_customer)) 
        
        
     
        if len(queue_w_customer)<101:
            if error>1 :
                print('self',env.now - self.arrival_t)
                if (env.now - self.arrival_t)<10:
                    yield env.timeout(random.expovariate(CALL_RATE))
                    if prob<3:
                        queue_w_operator1.append(self.env.now - self.arrival_t)
                        yield self.env.process(self.call1())
                        
                    if prob>=3:  
                        queue_w_operator2.append(self.env.now - self.arrival_t)
                        yield self.env.process(self.call2())
                        
                else:
                    print('%s A caller hangs up-10 min- at %g' % (self.name, self.env.now))
            else:
                 print('%s A caller hangs up at %g' % (self.name, self.env.now))
        else:
            print('%s A caller drops without answering.-100 parallel channels- at %g' % (self.name, self.env.now))
            queue_w_customer.pop()
            print('drop len',len(queue_w_customer)) 


            
    def call1(self):
        
        with operator1.request() as req:
            queue_w_customer.pop()
            print('op1 len',len(queue_w_customer))
            yield req
            print('%s is assigned to a operator1 at %g' % (self.name, self.env.now))
            queue_w_times_operator1.append(self.env.now - self.arrival_t)
            yield self.env.process(self.get_help1_operator1())
            print('%s is done operator1 at %g' % (self.name, self.env.now))
            self.end_time = self.env.now
            queue_w_operator1.pop()
            

    def call2(self):
        
        with operator2.request() as req:
            yield req
            queue_w_customer.pop()
            print('op2 len',len(queue_w_customer))
            print('%s is assigned to a operator2 at %g' % (self.name, self.env.now))
            queue_w_times_operator2.append(self.env.now - self.arrival_t)
            yield self.env.process(self.get_help2_operator2())
            print('%s is done operator2 at %g' % (self.name, self.env.now))
            self.end_time = self.env.now
            queue_w_operator2.pop()

            

    def get_help1_operator1(self):
        duration = numpy.random.lognormal(mu, sigma)
        while duration < 0:
            duration = numpy.random.lognormal(mu, sigma)

        yield self.env.timeout(duration)
        service_times_operator1.append(duration)
        
    def get_help2_operator2(self):
        duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
        while duration < 0:
            duration = random.gauss(SERVICE_TIME_OPERATOR2_MEAN, SERVICE_TIME_OPERATOR2_DEVIATION)
            
        yield self.env.timeout(duration)
        service_times_operator2.append(duration)



In [270]:

def customer_generator(env, operator1, operator2):
    """Generate new customer that call to call center."""
    for i in range(CUSTOMER_COUNT):
        yield env.timeout(random.expovariate(INTERARRIVAL_RATE))
        customer = Customer('Customer %s' %(i+1), env, operator1, operator2)
        

In [271]:
def break_generator1(env, operator1):
    """Generate new break for operator1."""
    BREAK_TIME_COUNTER1 = 0
    
    while True:
      
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        print('waiting1', len(queue_w_operator1))

        with operator1.request() as req:
            yield req
            print('Operator1 gives break at %g' % (env.now))
            yield env.timeout(BREAK_TIME)
            print('Operator1 break done at %g' % (env.now))
            BREAK_TIME_COUNTER1+=1
            print('break1',BREAK_TIME_COUNTER1)

      

In [272]:
def break_generator2(env, operator2):
    """Generate new break for operator2."""
    
    BREAK_TIME_COUNTER2 = 0
    
    while True:
        
           
        yield env.timeout(random.expovariate(BREAK_TIME_RATE))
        print('waiting2', len(queue_w_operator2))

      
        with operator2.request() as req:
            yield req
            print('Operator2 gives break at %g' % (env.now))
            yield env.timeout(BREAK_TIME)
            print('Operator2 break done at %g' % (env.now))
            BREAK_TIME_COUNTER2+=1
            print('break2',BREAK_TIME_COUNTER2)

In [273]:
env = simpy.Environment()
operator1 = simpy.Resource(env, capacity = 1)
operator2 = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env, operator1, operator2))
env.process(break_generator1(env, operator1))
env.process(break_generator2(env, operator2))
env.run(until=1200)

Customer 1 initiated a call at 3.31508
prob 5.993637605653995
error 3.381957220400075
inital len 1
self 0.0
op2 len 0
Customer 1 is assigned to a operator2 at 5.57578
Customer 2 initiated a call at 6.18385
prob 5.460343597252942
error 9.858652929789429
inital len 1
self 0.0
Customer 3 initiated a call at 8.65602
prob 3.452484526404228
error 1.342065909993051
inital len 2
self 0.0
Customer 1 is done operator2 at 10.1254
op2 len 1
Customer 2 is assigned to a operator2 at 10.1254
Customer 2 is done operator2 at 13.895
op2 len 0
Customer 3 is assigned to a operator2 at 15.646
waiting1 0
Operator1 gives break at 17.5986
Customer 4 initiated a call at 17.6103
prob 6.889792730135622
error 4.527166508287751
inital len 1
self 0.0
Operator1 break done at 20.5986
break1 1
Customer 3 is done operator2 at 21.9859
Customer 5 initiated a call at 26.4637
prob 9.440681562275989
error 8.91421358045429
inital len 2
self 0.0
op2 len 1
Customer 5 is assigned to a operator2 at 27.8388
Customer 6 initiated a

prob 0.12044766576482013
error 9.73182371407085
inital len 11
self 0.0
Customer 124 is done operator2 at 734.464
op2 len 10
Customer 125 is assigned to a operator2 at 734.464
op1 len 9
Customer 125 is done operator2 at 739.009
Customer 127 initiated a call at 742.569
prob 0.024089835718144048
error 4.772254165542067
inital len 10
self 0.0
op1 len 9
Customer 128 initiated a call at 746.553
prob 1.5152117451204972
error 5.926035112796927
inital len 10
self 0.0
op1 len 9
Customer 129 initiated a call at 753.251
prob 5.522310009737515
error 2.55900205155653
inital len 10
self 0.0
op2 len 9
Customer 129 is assigned to a operator2 at 756.709
Customer 130 initiated a call at 760.107
prob 1.3295929055865807
error 4.981415740323723
inital len 10
self 0.0
Customer 129 is done operator2 at 763.577
op1 len 9
waiting2 0
Operator2 gives break at 776.394
Operator2 break done at 779.394
break2 10
Customer 131 initiated a call at 787.595
prob 6.997201093791894
error 1.1492578711630974
inital len 10
sel